# Use multiple `agents.json` files
In this example, we'll show how to use multiple `agents.json` files together to create an agent that posts GIFs to Twitter based on search terms


In [ ]:
%pip install agentsjson
%pip install openai

### Setup API keys and references to `agents.json` files
Make sure to enable "Read & Write" Permissions for Twitter Keys

In [27]:
TWITTER_BEARER_TOKEN="<your-twitter-bearer-token>"
TWITTER_CONSUMER_KEY="<your-twitter-consumer-key>"
TWITTER_CONSUMER_SECRET="<your-twitter-consumer-secret>"
TWITTER_ACCESS_TOKEN="<your-twitter-access-token>"
TWITTER_ACCESS_TOKEN_SECRET="<your-twitter-access-token-secret>"

GIPHY_API_KEY="<your-giphy-api-key>"
OPENAI_API_KEY="<your-openai-api-key>"
TWITTER_AGENTS_JSON_URL="https://raw.githubusercontent.com/wild-card-ai/agents-json/refs/heads/master/agents_json/twitter/agents.json"
GIPHY_AGENTS_JSON_URL="https://raw.githubusercontent.com/wild-card-ai/agents-json/refs/heads/master/agents_json/giphy/agents.json"

### Load the `agents.json` files

In [13]:
from agentsjson.core.models import Flow
from agentsjson.core.loader import load_agents_json

# load the agents.json file
twitter_bundle = load_agents_json(TWITTER_AGENTS_JSON_URL)
giphy_bundle = load_agents_json(GIPHY_AGENTS_JSON_URL)

# get the flows
twitter_flows = twitter_bundle.agentsJson.flows
giphy_flows = giphy_bundle.agentsJson.flows

### Define agent creation function

In [14]:
from openai import OpenAI
from agentsjson.core import ToolFormat, flows_tools, execute_flows

def create_agent(bundle, flows, system_prompt, auth):
    openai_client = OpenAI(api_key=OPENAI_API_KEY)
    
    def agent(user_request):
        response = openai_client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "system", "content": system_prompt}, {"role": "user", "content": user_request}],
            tools=flows_tools(flows, format=ToolFormat.OPENAI),
            tool_choice="auto",
        )
        
        # Run the flows
        response = execute_flows(response, format=ToolFormat.OPENAI, bundle=bundle, flows=flows, auth=auth)
        return response
    return agent

### Create Twitter and Giphy agents

In [15]:
from agentsjson.core.parsetools import flows_prompt

twitter_flows_context = flows_prompt(twitter_flows)
giphy_flows_context = flows_prompt(giphy_flows)

twitter_system_prompt = f"""You are an AI assistant that helps users interact with the Twitter API.
You have access to the following API flows:

{twitter_flows_context}

Analyze the user's request and use the appropriate API flows to accomplish the task.
You must give your arguments for the tool call as Structued Outputs JSON with keys `parameters` and `requestBody`"""

giphy_system_prompt = f"""You are an AI assistant that helps users interact with the Giphy API.
You have access to the following API flows:

{giphy_flows_context}

Analyze the user's request and call the corresponding tool call with the appropriate API flows to accomplish the task.
You must give your arguments for the tool call as Structued Outputs JSON with keys `parameters` and `requestBody`"""

In [16]:
from agentsjson.core.models.auth import ApiKeyAuthConfig, AuthType, OAuth1AuthConfig

twitter_auth = OAuth1AuthConfig(type=AuthType.OAUTH1, consumer_key=TWITTER_CONSUMER_KEY, consumer_secret=TWITTER_CONSUMER_SECRET, access_token=TWITTER_ACCESS_TOKEN, access_token_secret=TWITTER_ACCESS_TOKEN_SECRET)
giphy_auth = ApiKeyAuthConfig(type=AuthType.API_KEY, key_value=GIPHY_API_KEY)

twitter_agent = create_agent(twitter_bundle, twitter_flows, twitter_system_prompt, twitter_auth)
giphy_agent = create_agent(giphy_bundle, giphy_flows, giphy_system_prompt, giphy_auth)

### Option 1: Invoke the agents directly

In [ ]:
giphy_response = giphy_agent("find a random cat GIF that is funny")
response = twitter_agent(f"post a tweet with #agentsjson about a gif we found: {giphy_response}")

response

### Option 2: Create an agent to Orchestrate each Sub-Agent
Often a user's request is a single complex task that needs multiple sub-tasks to be accomplished.

We can create an agent that orchestrates the sub-agents to accomplish the task.


In [25]:
import json

agents_list = [
    {
        "name": "twitter_agent",
        "agent": twitter_agent,
        "description": "This agent makes requests to the Twitter API."
    },
    {
        "name": "giphy_agent",
        "agent": giphy_agent,
        "description": "This agent makes requests to the Giphy API."
    }
]

# Write a prompt for the orchestrator
system_prompt = """
You are a smart AI assistant that can help the user with their requests.
Break down the user's request into a list of sub-tasks and determine which agent to use and in what order to accomplish the task.
You have access to the following sub-agents with specialized capabilities:

{agents_descriptions}

Use the appropriate sub-agent tool call to accomplish the task. 
Make sure to pass a precise natural language description of the task to the sub-agent. Pass the right context from the previous sub-agent's responses to the next sub-agent through the task.

You'll be given a response from the previous sub-agent. Use that response to determine the next sub-task and the agent to use.
If there's no more sub-tasks to accomplish, return the word STOP.
"""
agents_descriptions = "\n".join([f"- {agent['name']}: {agent['description']}" for agent in agents_list])
formatted_system_prompt = system_prompt.format(agents_descriptions=agents_descriptions)

def agents_tools(agents_list):  
    return {
            "type": "function",
            "function": {
                "name": "execute_agent",
                "description": "This function is responsible for calling the appropriate agent to accomplish the task.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "agent_name": {
                            "type": "string",
                            "enum": [agent["name"] for agent in agents_list],
                            "description": "The name of the agent to call."
                        },
                        "task": {
                            "type": "string",
                            "description": "The task to accomplish described precisely in natural language."
                        }
                    },
                    "required": ["agent_name", "task"],
                }
            }
        }

def orchestrator(agents_list, messages):
    
    print("MESSAGES", messages)
        
    openai_client = OpenAI(api_key=OPENAI_API_KEY)
    
    # OpenAI function call to orchestrate the sub-agents
    response = openai_client.chat.completions.create(
        model="gpt-4o",  # Fixed model name
        messages=messages,
        tools=[agents_tools(agents_list)],
        tool_choice="auto",
        parallel_tool_calls=False
    )
    
    # Run all tool calls
    responses = []
    if not response.choices[0].message.tool_calls:
        return "STOP"
        
    for tool_call in response.choices[0].message.tool_calls:
        try:
            args = json.loads(tool_call.function.arguments)
            print("ARGS", args)
            task, agent_name = args["task"], args["agent_name"]
            
            agent = next((a for a in agents_list if a["name"] == agent_name), None)
            if agent:
                tool_response = agent["agent"](task)
                print("TOOL_RESPONSE", tool_response)
                responses.append(tool_response)
            else:
                raise ValueError(f"Agent {agent_name} not found")
        except json.JSONDecodeError:
            raise ValueError("Invalid tool call arguments format")
    
    # Process the responses
    final_response = json.dumps(responses[-1]) if responses else "STOP"
    if final_response != "STOP":
        next_responses = orchestrator(agents_list, messages + [{"role": "user", "content": final_response}])
        responses.extend(next_responses if isinstance(next_responses, list) else [next_responses])
    
    return responses

### Run Task

In [ ]:
query = "find one random GIF of a cat that is funny and share it on twitter with the hashtag #agentsjson"

orchestrator(agents_list, [{"role": "user", "content": query}])